In [1]:
import os
import json
import cv2
import numpy as np
import pandas as pd
import keras
import pandas as pd
import numpy as np
import os
from glob import glob
import itertools
import fnmatch
import random
import matplotlib.pylab as plt
import cv2
from scipy.misc import imresize, imread
import sklearn
from sklearn import model_selection
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, learning_curve, GridSearchCV
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import keras
from keras import layers
from tqdm import tqdm
from keras import backend as K
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, model_from_json,load_model,Model
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras.utils import layer_utils
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, UpSampling2D , Conv2D, MaxPool2D, MaxPooling2D,Input,Concatenate,GlobalAveragePooling2D,GlobalMaxPooling2D,ZeroPadding2D,AveragePooling2D,Reshape,merge,Convolution2D
%matplotlib inline
from keras import layers
from keras.applications import MobileNetV2
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
import tensorflow as tf
from tqdm import tqdm
import efficientnet.keras as efn 
from keras.layers import Dense
from keras.optimizers import Adam, Nadam
from keras_radam import RAdam
from keras import backend as K
from sklearn.utils import class_weight

Using TensorFlow backend.
/home/qali/anaconda3/envs/py37_gpu_env/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/qali/anaconda3/envs/py37_gpu_env/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/qali/anaconda3/envs/py37_gpu_env/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/qa

In [2]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [3]:
def get_model():
    input_img = Input(shape=(720, 720, 3))
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)



    input_img1 = Input(shape=(720, 720, 3))
    x1 = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img1)
    x1 = MaxPooling2D((2, 2), padding='same')(x1)
    x1 = Conv2D(8, (3, 3), activation='relu', padding='same')(x1)
    x1 = MaxPooling2D((2, 2), padding='same')(x1)
    x1 = Conv2D(8, (3, 3), activation='relu', padding='same')(x1)
    encoded1 = MaxPooling2D((2, 2), padding='same')(x1)


    combined = keras.layers.concatenate([encoded, encoded1])




    x = Conv2D(8, (3, 3), activation='relu', padding='same')(combined)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    
    
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(combined)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded1 = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    
    
    

    autoencoder = Model([input_img, input_img1], [decoded, decoded1])
    return autoencoder


In [4]:
def train_generator(training):
    k = 0
    while True:
        i = training[k]
        filename_f = '../BACH/Originals/'+i
        filename_p = '../BACH/Predictions_Mask/'+i
        filename_g = '../BACH/Ground_Truths/'+i      
        org = cv2.imread(filename_f)
        pred = cv2.imread(filename_p)
        ground = cv2.imread(filename_g)
#        print ("Train: ", filename_f)
#        print ("Train: ", filename_p)
#        print ("Train: ", filename_g)
        

        org = cv2.resize(org,(720, 720))/255.0
        pred = cv2.resize(pred,(720, 720))/255.0
        ground = cv2.resize(ground,(720, 720)) / 255.0
        ground1 = np.abs(1 - ground[:, :, 0])
        ground = ground[:, :, 0]
        
        
        org = org[np.newaxis, :]
        pred = pred[np.newaxis, :]
        
        X = [org, pred]
        Y = [ground[np.newaxis, :, :, np.newaxis], ground1[np.newaxis, :, :, np.newaxis]]
        yield X, Y   
        
        k += 1
        if k >= len(training):
            k = 0

In [5]:
def test_generator(testing):
    k = 0
    while True:
        i = testing[k]
        filename_f = 'Originals/'+i
        filename_p = 'Kwok_Predictions/'+i
        filename_g = 'Ground_Truths/'+i      
        org = cv2.imread(filename_f)
        pred = cv2.imread(filename_p)
        ground = cv2.imread(filename_g)
#        print ("Test: ", filename_f)
#        print ("Test: ", filename_p)
#        print ("Test: ", filename_g)
        
        org = cv2.resize(org,(720, 720))/255.0
        pred = cv2.resize(pred,(720, 720))/255.0
        ground = cv2.resize(ground,(720, 720)) / 255.0
        ground1 = np.abs(1 - ground[:, :, 0])
        ground = ground[:, :, 0]
        
        org = org[np.newaxis, :]
        pred = pred[np.newaxis, :]
        
        X = [org, pred]
        Y = [ground[np.newaxis, :, :, np.newaxis], ground1[np.newaxis, :, :, np.newaxis]]

        yield X, Y 
        
        k += 1
        if k >= len(testing):
            k = 0

# Rename Directories If needed. I did that....!!!!

In [6]:
all_files = os.listdir('../BACH/Ground_Truths/')
all_files

['A08.png',
 'A02.png',
 'A10.png',
 'A01.png',
 'A07.png',
 'A05.png',
 'A03.png',
 'A09.png',
 'A06.png',
 'A04.png']

In [7]:
test_files = os.listdir('Originals')
test_files

['A0BO.png', 'A0BD.png', 'A0CP.png', 'A0CQ.png']

In [8]:
autoencoder = get_model()
#autoencoder.compile(optimizer=RAdam(learning_rate=0.0005), loss='binary_crossentropy', metrics=[dice_coef])
autoencoder.compile(optimizer=RAdam(learning_rate=0.0005), loss=dice_coef_loss, metrics=[dice_coef])

train_gen = train_generator(all_files)
test_gen = test_generator(test_files)
checkpoint = ModelCheckpoint('Kwok_Dice_Coeff_Complete_All_DCE.h5', monitor='val_loss', verbose=1, save_best_only=True, 
                             save_weights_only=False,mode='min')
autoencoder.fit_generator(train_gen,steps_per_epoch = 10,epochs=70
        , validation_data=test_gen,
    validation_steps=1, callbacks=[checkpoint])

Epoch 1/70
10/10 [==============================] - 3s 302ms/step - loss: -0.9112 - conv2d_10_loss: -0.6240 - conv2d_14_loss: -0.2872 - conv2d_10_dice_coef: 0.6240 - conv2d_14_dice_coef: 0.2872 - val_loss: -0.7614 - val_conv2d_10_loss: -0.7010 - val_conv2d_14_loss: -0.0604 - val_conv2d_10_dice_coef: 0.7010 - val_conv2d_14_dice_coef: 0.0604

Epoch 00001: val_loss improved from inf to -0.76140, saving model to Kwok_Dice_Coeff_Complete_All_DCE.h5
Epoch 2/70
10/10 [==============================] - 0s 33ms/step - loss: -0.9190 - conv2d_10_loss: -0.6314 - conv2d_14_loss: -0.2876 - conv2d_10_dice_coef: 0.6314 - conv2d_14_dice_coef: 0.2876 - val_loss: -0.8735 - val_conv2d_10_loss: -0.6927 - val_conv2d_14_loss: -0.1808 - val_conv2d_10_dice_coef: 0.6927 - val_conv2d_14_dice_coef: 0.1808

Epoch 00002: val_loss improved from -0.76140 to -0.87348, saving model to Kwok_Dice_Coeff_Complete_All_DCE.h5
Epoch 3/70
10/10 [==============================] - 0s 33ms/step - loss: -0.9336 - conv2d_10_loss: -

10/10 [==============================] - 0s 33ms/step - loss: -1.6927 - conv2d_10_loss: -0.9414 - conv2d_14_loss: -0.7513 - conv2d_10_dice_coef: 0.9414 - conv2d_14_dice_coef: 0.7513 - val_loss: -1.3492 - val_conv2d_10_loss: -0.9506 - val_conv2d_14_loss: -0.3986 - val_conv2d_10_dice_coef: 0.9506 - val_conv2d_14_dice_coef: 0.3986

Epoch 00040: val_loss did not improve from -1.80161
Epoch 41/70
10/10 [==============================] - 0s 32ms/step - loss: -1.6935 - conv2d_10_loss: -0.9416 - conv2d_14_loss: -0.7519 - conv2d_10_dice_coef: 0.9416 - conv2d_14_dice_coef: 0.7519 - val_loss: -1.8112 - val_conv2d_10_loss: -0.9916 - val_conv2d_14_loss: -0.8196 - val_conv2d_10_dice_coef: 0.9916 - val_conv2d_14_dice_coef: 0.8196

Epoch 00041: val_loss improved from -1.80161 to -1.81124, saving model to Kwok_Dice_Coeff_Complete_All_DCE.h5
Epoch 42/70
10/10 [==============================] - 0s 33ms/step - loss: -1.6941 - conv2d_10_loss: -0.9417 - conv2d_14_loss: -0.7523 - conv2d_10_dice_coef: 0.9417 

In [8]:
def dice(im1, im2, empty_score=1.0, THRESH=0.5):
    """
    Computes the Dice coefficient, a measure of set similarity.
    Parameters
    ----------
    im1 : array-like, bool
        Any array of arbitrary size. If not boolean, will be converted.
    im2 : array-like, bool
        Any other array of identical size. If not boolean, will be converted.
    Returns
    -------
    dice : float
        Dice coefficient as a float on range [0,1].
        Maximum similarity = 1
        No similarity = 0
        Both are empty (sum eq to zero) = empty_score
        
    Notes
    -----
    The order of inputs for `dice` is irrelevant. The result will be
    identical if `im1` and `im2` are switched.
    """
    im1[im1 >= THRESH] = 1
    im1[im1 < THRESH] = 0
    im2[im2 >= THRESH] = 1
    im2[im2 < THRESH] = 0

    
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    im_sum = im1.sum() + im2.sum()
    if im_sum == 0:
        return empty_score

    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)
#     print ("###############################")
#     print ("INTERSECTION", intersection.sum())
#     print ("Predictions 1s", im1.sum())
#     print ("Ground Truths 1s", im2.sum())
#     print ("SUM OF BOTHS", im2.sum() + im1.sum())
    dc = 2. * intersection.sum() / im_sum
    print ("DICE : ", dc)
    return dc

In [9]:
model = load_model('Kwok_Dice_Coeff_Complete_All_DCE.h5', compile=False)
total = 0
xtotal = 0
test_files = os.listdir('Originals')
for each in test_files:
    print (each)

    filename_t = 'Kwok_TCGA_Stage_2_DICE_LOSS/'+each
    filename_xt = 'Kwok_TCGA_Stage_2_DICE_LOSS/C_'+each
    filename_f = 'Originals/'+each
    filename_p = 'Kwok_Predictions/'+each
    filename_g = 'Ground_Truths/'+each
    
    
    
    org = cv2.imread(filename_f)
    org = cv2.resize(org,(720, 720))
    pred = cv2.imread(filename_p)
    pred = cv2.resize(pred,(720, 720))    
    gr = cv2.imread(filename_g)
    gr = cv2.resize(gr,(720, 720))    
    
    org = org[np.newaxis, :]
    pred = pred[np.newaxis, :]
    gr = gr[:, :, 0]
    
    
    X = [org, pred]
    P = model.predict(X, verbose=1)
    #print (P[0][P[0] != P[1]].shape)
    
    pred = P[0][0, :, :, 0]
#    pred[pred >= 0.5] = 1
#    pred[pred < 0.5] = 0

    pred1 = P[1][0, :, :, 0]
#    pred1[pred1 >= 0.5] = 1
#    pred1[pred1 < 0.5] = 0

#     print ("BRANCH 1")
#     dice(pred, gr)
#     print ("BRANCH 2")
#     dice(np.abs(pred1), np.abs(1-gr))
#     print ("COMBINING")
    combined_prob = (pred + np.abs(1-pred1)) / 2.0
    total = total + dice(combined_prob, gr)
    xtotal = xtotal + dice(np.abs(1-combined_prob), np.abs(1-gr))
    
    
#    pred[pred == 0]  = 255.0


    pred = np.abs(1-pred)
    pred[pred == 1]  = 255.0
    
    prediction = np.zeros((720,720,3))
    prediction[:,:,0]= pred
    prediction[:,:,1] = pred
    prediction[:,:,2] = pred
    cv2.imwrite(filename_t, prediction)
    
    
    
    pred = combined_prob.copy()
    pred = np.abs(1-pred)
    pred[pred == 0]  = 255.0
    pred[pred == 1]  = 0
    
    
    prediction = np.zeros((720,720,3))
    prediction[:,:,0]= pred
    prediction[:,:,1] = pred
    prediction[:,:,2] = pred
    cv2.imwrite(filename_xt, prediction)
    
    

A0BO.png
1/1 [==============================] - 1s 750ms/step
DICE :  0.9923784359044543
DICE :  0.6890295528766681
A0BD.png
1/1 [==============================] - 0s 16ms/step
DICE :  0.9691713907151482
DICE :  0.6448753863987635
A0CP.png
1/1 [==============================] - 0s 16ms/step
DICE :  0.9480018849741241
DICE :  0.4821798503361096
A0CQ.png
1/1 [==============================] - 0s 18ms/step
DICE :  0.9544045419865749
DICE :  0.2946642733467825


In [10]:
total / 4.0, xtotal / 4.0

(0.9659890633950754, 0.5276872657395809)